Classificador de reviews de jogos da steam.

As reviews são compostas por texto e um label (

In [3]:
import pandas as pd
from matplotlib import pyplot as plt

# Limpando o Dataset

Carregando os dados

In [4]:
df = pd.read_csv('SteamReviews.csv')

In [5]:
df.describe()

,app_id,review_score,review_votes
count,6.417106e+06,6.417106e+06,6.417106e+06
mean,2.274695e+05,6.394992e-01,1.472446e-01
std,1.260451e+05,7.687918e-01,3.543496e-01
min,1.000000e+01,-1.000000e+00,0.000000e+00
25%,2.018100e+05,1.000000e+00,0.000000e+00
50%,2.391600e+05,1.000000e+00,0.000000e+00
75%,3.056200e+05,1.000000e+00,0.000000e+00
max,5.653400e+05,1.000000e+00,1.000000e+00


Escolhendo uma amostra do dataset

In [6]:
df = df.sample(n = 5000)

In [7]:
df = df[['review_text', 'review_score']]

Checando elementos duplicados

In [8]:
df.duplicated().sum()

835

Removendo duplicatas

In [9]:
df.drop_duplicates(inplace=True)

Renomeando colunas

In [10]:
df.rename(columns={'review_text':'text', 'review_score':'target'}, inplace=True)

In [11]:
df

,text,target
1086545,Fun and very colorful,1
2082368,Best PC platforming-sidescroller of the centur...,1
2741153,Early Access Review,1
5795239,"Epic siege battles and land battles, fixed alm...",1
3401389,Very few games take me back to my Super NES da...,1
...,...,...
4697904,https://youtu.be/pe14MQEL5LU Has science go...,1
1446768,wololo 10/10 would convert again,1
3062769,this game is fun now i can play turbo dismount...,1
826517,"Hey buddy, would you like to play a dead call ...",1


In [12]:
df.reset_index(drop=True, inplace=True)

## Removendo valores faltantes

In [13]:
missing_values = df.isna().sum()
missing_values

text      2
target    0
dtype: int64

In [14]:
df.dropna(inplace=True)

In [15]:
missing_values = df.isna().sum()
missing_values

text      0
target    0
dtype: int64

## Mudando de rótulos -1 e +1 para 0 e 1

In [16]:
df["target"].unique()

array([ 1, -1], dtype=int64)

In [17]:
df["target"] = df["target"].replace({-1:0, 1: 1})

# Pré Processamento

Imports

In [18]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuário\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuário\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Definindo função de conversão de texto:

Torna mínusculas, tokenizadas, PorterStemmer e sem stopwords e pontuações

In [19]:
def convert_text(text):
  text = text.lower()
  text = nltk.word_tokenize(text)

  x = []
  y = []

  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      x.append(i)
  for i in x:
    y.append(ps.stem(i))
  return ' '.join(y)

In [20]:
df['converted_text'] = df['text'].apply(convert_text)

In [21]:
df['converted_text']

0                                               fun color
1       best pc platforming-sidescrol centuri buy even...
2                                     earli access review
3       epic sieg battl land battl fix almost problem ...
4       game take back super ne day titl almost flawle...
                              ...                        
4160    http //youtu.be/pe14mqel5lu scienc gone far in...
4161                           wololo 10/10 would convert
4162    game fun play turbo dismount comput laptop pad...
4163    hey buddi would like play dead call duti repli...
4164    's certainli great parodi plethora simul game ...
Name: converted_text, Length: 4163, dtype: object

## Aplicando vetorizador
Contador de palavras

In [22]:
target = df.target

In [23]:
reviews = df.converted_text

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [25]:
X_train_counts = count_vect.fit_transform(reviews)

In [26]:
X_train_counts

<4163x11524 sparse matrix of type '<class 'numpy.int64'>'
	with 117077 stored elements in Compressed Sparse Row format>

## Transformando em tf-idf
Destacar palavras-chave e reduzir o peso de palavras comuns que aparecem em muitos documentos

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer

In [28]:
tfidf_transformer = TfidfTransformer()

In [29]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Modelo

Multinomial Naive-Bayes

## Classificador

In [30]:
from sklearn.naive_bayes import MultinomialNB

In [31]:
clf = MultinomialNB()

## Teste

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, target, test_size=0.2, random_state=34)

In [33]:
from sklearn.metrics import accuracy_score


In [34]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8331332533013205